In [ ]:
#hide
from pySplitting.core import *
from sympy import *

# pySplitting

> A project to have python code that generates the order conditions for exponential splitting methods with the equations of M. Thalhammer given in the 2008 paper "High-Order Exponential Operator Splitting Methods for Time-Dependent Schrödinger Equations" available at ´https://epubs.siam.org/doi/abs/10.1137/060674636?mobileUi=0´


This repo is binder enabled: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/CaptainSifff/pySplitting/master)

## Exposition
Exponential Splitting methods are an a numerical technique which can for instance be used for solving ordinary differential equations:
$$
\dot{y}(\tau) = -H y(\tau)
$$
If $H = T + V$ , i. e. has a substructure with two components we can approximate the solution as 
$$
y(\tau) = e^{-H \tau} = e^{-T \tau } e^{-V \tau} + \mathcal{O}(\tau^2)
$$
We can achieve higher order methods by using more expontials as in
$$
y(t) = \prod_i e^{t_i T \tau} e^{v_i V \tau} + \mathcal{O}(\tau^{p+1})
$$
Determining the systems of polynomials that $t_i$ and $v_i$ have to fulfill is the purpose of this library/notebook.

## How to use

This is a python project that generates the equations for the order conditions such that exponential splitting methods of the given order or with the desired features can be designed. To that end we utilize sympy to get symbolic expressions that can be further processed.

First we will derive the equations for the familiar leapfrog/Strang splitting. This is a method of order p=2 and uses the indeterminates ((t0,t1),(v0,v1))=((1/2,1/2).(1,0)). Hence we have to set up sympy first to give us the respective symbols:

In [ ]:
from sympy.matrices.dense import symarray
from sympy import *
tvec=[*symarray('t',2)]
vvec=[*symarray('v',2)]

Next we build the equations. To that end we use the function CreateCondition().
The first argument is the order, the second and third argument are the indeterminates for the resulting polynomials.

In [ ]:
strangconds=CreateConditions(2,tvec,vvec)
display(strangconds)

[Eq(t_0 + t_1, 1), Eq(-v_0 - v_1 + 1, 0), Eq(-t_0*v_0 - v_1 + 1/2, 0)]

Eq(a,b) is sympy syntax for the equality a = b. Now we check wether the solutions fulfill these equations:

In [ ]:
[it.subs([('t_0',sympify(1)/2),('t_1',sympify(1)/2),('v_0',1),('v_1',0)]) for it in strangconds]

[True, True, True]

It works! Now we move on to check the improved Leapfrog by McLachlan(McLachlan 1995, https://www.massey.ac.nz/~rmclachl/sisc95.pdf), and utilize that we can impose already some structure(symmetry and the consistency equations) on the indeterminates to get some simplifications:

In [ ]:
tvec=[*symarray('t',3)]
tvec[2]=tvec[0]
tvec[1]=1-2*tvec[0]
vvec=[*symarray('v',3)]
vvec[2]=0
vvec[1]=vvec[0]
print("Indeterminates: ",tvec,vvec)
McLachlanconds=CreateConditions(2,tvec,vvec)
print(McLachlanconds)

Indeterminates:  [t_0, 1 - 2*t_0, t_0] [v_0, v_0, 0]
[True, Eq(1 - 2*v_0, 0), Eq(-t_0*v_0 - v_0*(1 - t_0) + 1/2, 0)]


We check that the given numbers(v_0=1/2,t_0=0.1931) satisfy the equation:

In [ ]:
[it.subs([('t_0',0.19318332),('v_0',sympify(1)/2)]) for it in McLachlanconds]

[True, True, True]

True! Now we move on to two more fourth order examples from the literature:
First the fourth order method from McLachlan's 1995 paper:

In [ ]:
tvec=[*symarray('t',5)]
tvec[4]=tvec[0]
tvec[3]=tvec[1]
tvec[2]=1-2*(tvec[0]+tvec[1])
vvec=[*symarray('v',5)]
vvec[4]=0
vvec[3]=vvec[0]
vvec[1]=(1-2*vvec[0])/2
vvec[2]=vvec[1]
print("Indeterminates: ",tvec,vvec)
McLachlanconds4=CreateConditions(4,tvec,vvec)
print(McLachlanconds4)
[it.subs([('v_0',S(6)/S(11)),('t_0',(642+Pow(471,S(1)/2))/S(3924)),('t_1',S(121)/S(3924)*(12-Pow(471,S(1)/2)))]) for it in McLachlanconds4]

Indeterminates:  [t_0, t_1, -2*t_0 - 2*t_1 + 1, t_1, t_0] [v_0, 1/2 - v_0, 1/2 - v_0, v_0, 0]
[True, True, Eq(-t_0*v_0 - v_0*(1 - t_0) - (1/2 - v_0)*(t_0 + t_1) - (1/2 - v_0)*(-t_0 - t_1 + 1) + 1/2, 0), Eq(-t_0**2*v_0 - v_0*(1 - t_0)**2 - (1/2 - v_0)*(t_0 + t_1)**2 - (1/2 - v_0)*(-t_0 - t_1 + 1)**2 + 1/3, 0), Eq(-t_0**3*v_0 - v_0*(1 - t_0)**3 - (1/2 - v_0)*(t_0 + t_1)**3 - (1/2 - v_0)*(-t_0 - t_1 + 1)**3 + 1/4, 0), Eq(-3*t_0*v_0**2/2 - 2*t_0*v_0*(1/2 - v_0) - v_0**2*(1 - t_0)/2 - v_0*(1/2 - v_0)*(t_0 + t_1) - v_0*(1/2 - v_0)*(-t_0 - t_1 + 1) - 3*(1/2 - v_0)**2*(t_0 + t_1)/2 - (1/2 - v_0)**2*(-t_0 - t_1 + 1)/2 + 1/6, 0), Eq(-3*t_0**2*v_0**2/2 - 2*t_0**2*v_0*(1/2 - v_0) - v_0**2*(1 - t_0)**2/2 - v_0*(1/2 - v_0)*(t_0 + t_1)**2 - v_0*(1/2 - v_0)*(-t_0 - t_1 + 1)**2 - 3*(1/2 - v_0)**2*(t_0 + t_1)**2/2 - (1/2 - v_0)**2*(-t_0 - t_1 + 1)**2/2 + 1/12, 0), Eq(-7*t_0*v_0**3/6 - 3*t_0*v_0**2*(1/2 - v_0) - 2*t_0*v_0*(1/2 - v_0)**2 - v_0**3*(1 - t_0)/6 - v_0**2*(1/2 - v_0)*(t_0 + t_1)/2 - v_0**2*(1/

[True, True, True, True, True, True, True, True]

Next we check the popular S_6 4 method by from Blanes2002("Practical symplectic partitioned Runge–Kutta and Runge–Kutta–Nyström methods")
Since it's a symmetric method it is sufficient to work with the third order equations.

In [ ]:
tvec=[*symarray('t',7)]
tvec[6]=tvec[0]
tvec[5]=tvec[1]
tvec[4]=tvec[2]
tvec[3]=1-2*(tvec[0]+tvec[1]+tvec[2])
vvec=[*symarray('v',7)]
vvec[2]=(1-2*(vvec[0]+vvec[1]))/2
vvec[6]=0
vvec[5]=vvec[0]
vvec[4]=vvec[1]
vvec[3]=vvec[2]
print("Indeterminates: ",tvec,vvec)
S64conds=CreateConditions(3,tvec,vvec)
display(S64conds)

Indeterminates:  [t_0, t_1, t_2, -2*t_0 - 2*t_1 - 2*t_2 + 1, t_2, t_1, t_0] [v_0, v_1, -v_0 - v_1 + 1/2, -v_0 - v_1 + 1/2, v_1, v_0, 0]


[True,
 True,
 Eq(-t_0*v_0 - v_0*(1 - t_0) - v_1*(t_0 + t_1) - v_1*(-t_0 - t_1 + 1) - (t_0 + t_1 + t_2)*(-v_0 - v_1 + 1/2) - (-v_0 - v_1 + 1/2)*(-t_0 - t_1 - t_2 + 1) + 1/2, 0),
 Eq(-t_0**2*v_0 - v_0*(1 - t_0)**2 - v_1*(t_0 + t_1)**2 - v_1*(-t_0 - t_1 + 1)**2 - (t_0 + t_1 + t_2)**2*(-v_0 - v_1 + 1/2) - (-v_0 - v_1 + 1/2)*(-t_0 - t_1 - t_2 + 1)**2 + 1/3, 0),
 Eq(-3*t_0*v_0**2/2 - 2*t_0*v_0*v_1 - 2*t_0*v_0*(-v_0 - v_1 + 1/2) - v_0**2*(1 - t_0)/2 - v_0*v_1*(t_0 + t_1) - v_0*v_1*(-t_0 - t_1 + 1) - v_0*(t_0 + t_1 + t_2)*(-v_0 - v_1 + 1/2) - v_0*(-v_0 - v_1 + 1/2)*(-t_0 - t_1 - t_2 + 1) - 3*v_1**2*(t_0 + t_1)/2 - v_1**2*(-t_0 - t_1 + 1)/2 - 2*v_1*(t_0 + t_1)*(-v_0 - v_1 + 1/2) - v_1*(t_0 + t_1 + t_2)*(-v_0 - v_1 + 1/2) - v_1*(-v_0 - v_1 + 1/2)*(-t_0 - t_1 - t_2 + 1) - 3*(t_0 + t_1 + t_2)*(-v_0 - v_1 + 1/2)**2/2 - (-v_0 - v_1 + 1/2)**2*(-t_0 - t_1 - t_2 + 1)/2 + 1/6, 0)]

And now let's see whether it works. For precision reasons a simple true/false output is not sufficient, hence we ask for the residual.

In [ ]:
simpleS64conds=[it.simplify() for it in S64conds]
print(simpleS64conds)
res=simpleS64conds[:3]+[it.lhs.subs([
    ('t_0',S(0.0792036964311956)),
          ('t_1',S(0.35317290604977410)),
          ('t_2',S(-0.0420650803577195)),
          ('v_0',S(0.2095151066133620)),
          ('v_1',S(-0.143851773179818))
         ]) for it in simpleS64conds[3:]]
print(res)

[True, True, True, Eq(t_0**2*v_0 + v_0*(t_0 - 1)**2 + v_1*(t_0 + t_1)**2 + v_1*(t_0 + t_1 - 1)**2 - (t_0 + t_1 + t_2)**2*(v_0 + v_1 - 1/2) - (v_0 + v_1 - 1/2)*(t_0 + t_1 + t_2 - 1)**2 - 1/3, 0), Eq(t_0/4 + t_1*v_0**2 - t_1*v_0 + t_1/4 + t_2*v_0**2 + 2*t_2*v_0*v_1 - t_2*v_0 + t_2*v_1**2 - t_2*v_1 + t_2/4 - 1/24, 0)]
[True, True, True, -4.33680868994202e-19, 3.79470760369927e-18]


We see that the first and second order equations are already fulfilled. For the others we see that the residual is numerically vanishing.

## Complex Hermitian Methods where one set of coefficients is real

As outlined in [arXiv:2009.04491](https://arxiv.org/abs/2009.04491) certain quantum Monte Carlo methods
necessitate special hermitian methods and the new family of complex hermitian methods with one real set of coefficients was introduced.
In this section we check our new hermitian methods where one set of coefficients is real.

### CHR_3 3

In [ ]:
avec=[*symarray('a',4)]
bvec=[*symarray('b',4)]
tvec=[
    avec[0]+I*bvec[0],
    avec[1]+I*bvec[1],
    avec[1]-I*bvec[1],
    avec[0]-I*bvec[0],
]
vvec=[*symarray('v',4)]
vvec[1]=1-2*vvec[0]
vvec[3]=0
vvec[2]=vvec[0]
print("Indeterminates: ",tvec,vvec)
CHR33conds=CreateConditions(3,tvec,vvec)
display(CHR33conds)

Indeterminates:  [a_0 + I*b_0, a_1 + I*b_1, a_1 - I*b_1, a_0 - I*b_0] [v_0, 1 - 2*v_0, v_0, 0]


[Eq(2*a_0 + 2*a_1, 1),
 True,
 Eq(-v_0*(a_0 + I*b_0) - v_0*(a_0 + 2*a_1 + I*b_0) - (1 - 2*v_0)*(a_0 + a_1 + I*b_0 + I*b_1) + 1/2, 0),
 Eq(-v_0*(a_0 + I*b_0)**2 - v_0*(a_0 + 2*a_1 + I*b_0)**2 - (1 - 2*v_0)*(a_0 + a_1 + I*b_0 + I*b_1)**2 + 1/3, 0),
 Eq(-3*v_0**2*(a_0 + I*b_0)/2 - v_0**2*(a_0 + 2*a_1 + I*b_0)/2 - v_0*(1 - 2*v_0)*(a_0 + I*b_0) - v_0*(1 - 2*v_0)*(a_0 + a_1 + I*b_0 + I*b_1) - (1 - 2*v_0)**2*(a_0 + a_1 + I*b_0 + I*b_1)/2 + 1/6, 0)]

In [ ]:
res=[it.simplify().subs([
    ('a_0',3*S(1)/S(24)),
          ('a_1',3*S(1)/S(8)),
      ('b_0',-Pow(3,S(1)/S(2))/S(24)),
          ('b_1',Pow(3,S(1)/S(2))/S(8)),
          ('v_0',S(1)/S(3))
         ]).simplify() for it in CHR33conds]
print(res)

[True, True, True, True, True]


### CHR_5 4

In [ ]:
avec=[*symarray('a',6)]
bvec=[*symarray('b',6)]
tvec=[
    avec[0]+I*bvec[0],
    avec[1]+I*bvec[1],
    avec[2]+I*bvec[2],
    avec[2]-I*bvec[2],
    avec[1]-I*bvec[1],
    avec[0]-I*bvec[0]
]
vvec=[*symarray('v',6)]
vvec[5]=0
vvec[4]=vvec[0]
vvec[3]=vvec[1]
vvec[2]=1-2*(vvec[0]+vvec[1])
print("Indeterminates: ",tvec,vvec)
CHR54conds=CreateConditions(4,tvec,vvec)
#display(CHR54conds)

Indeterminates:  [a_0 + I*b_0, a_1 + I*b_1, a_2 + I*b_2, a_2 - I*b_2, a_1 - I*b_1, a_0 - I*b_0] [v_0, v_1, -2*v_0 - 2*v_1 + 1, v_1, v_0, 0]


In [ ]:
simpleCHR54conds=[it.simplify() for it in CHR54conds]
simpleCHR54conds[0],simpleCHR54conds[1] = simpleCHR54conds[1],simpleCHR54conds[0] # swap elements for easier indexing
print(simpleCHR54conds)
res=[simpleCHR54conds[0]]+[(it.lhs-it.rhs).subs([
    ('a_0',S(0.07703277090337929685)),
    ('a_1',S(0.20575040027785073837)),
    ('a_2',S(0.21721682881876996478)),
    ('b_0',S(-0.018324799648727332496)),
    ('b_1',S(0.083675123976098550021)),
    ('b_2',S(-0.152490283366744769770)),
    ('v_0',S(0.18995702345368160876)),
    ('v_1',S(1)/S(5))
         ]).simplify() for it in simpleCHR54conds[1:]]
print(res)

[True, Eq(a_0 + a_1 + a_2, 1/2), Eq(a_0 + a_1 + a_2 + I*b_0 - 2*I*b_1*v_0 + I*b_1 - 2*I*b_2*v_0 - 2*I*b_2*v_1 + I*b_2 - 1/2, 0), Eq(v_0*(a_0 + I*b_0)**2 + v_0*(a_0 + 2*a_1 + 2*a_2 + I*b_0)**2 + v_1*(a_0 + a_1 + I*b_0 + I*b_1)**2 + v_1*(a_0 + a_1 + 2*a_2 + I*b_0 + I*b_1)**2 - (2*v_0 + 2*v_1 - 1)*(a_0 + a_1 + a_2 + I*b_0 + I*b_1 + I*b_2)**2 - 1/3, 0), Eq(v_0*(a_0 + I*b_0)**3 + v_0*(a_0 + 2*a_1 + 2*a_2 + I*b_0)**3 + v_1*(a_0 + a_1 + I*b_0 + I*b_1)**3 + v_1*(a_0 + a_1 + 2*a_2 + I*b_0 + I*b_1)**3 - (2*v_0 + 2*v_1 - 1)*(a_0 + a_1 + a_2 + I*b_0 + I*b_1 + I*b_2)**3 - 1/4, 0), Eq(a_0/2 + a_1*v_0**2 - a_1*v_0 + a_1/2 + a_2*v_0**2 + 2*a_2*v_0*v_1 - a_2*v_0 + a_2*v_1**2 - a_2*v_1 + a_2/2 + I*b_0/2 - I*b_1*v_0 + I*b_1/2 - I*b_2*v_0 - I*b_2*v_1 + I*b_2/2 - 1/6, 0), Eq(3*v_0**2*(a_0 + I*b_0)**2/2 + v_0**2*(a_0 + 2*a_1 + 2*a_2 + I*b_0)**2/2 + 2*v_0*v_1*(a_0 + I*b_0)**2 + v_0*v_1*(a_0 + a_1 + I*b_0 + I*b_1)**2 + v_0*v_1*(a_0 + a_1 + 2*a_2 + I*b_0 + I*b_1)**2 - v_0*(a_0 + I*b_0)**2*(2*v_0 + 2*v_1 - 1) -

### CHR_9 5

In [ ]:
avec=[*symarray('a',10)]
bvec=[*symarray('b',10)]
tvec=[
    avec[0]+I*bvec[0],
    avec[1]+I*bvec[1],
    avec[2]+I*bvec[2],
    avec[3]+I*bvec[3],
    avec[4]+I*bvec[4],
    avec[4]-I*bvec[4],
    avec[3]-I*bvec[3],
    avec[2]-I*bvec[2],
    avec[1]-I*bvec[1],
    avec[0]-I*bvec[0]
]
vvec=[*symarray('v',10)]
vvec[9]=0
vvec[8]=vvec[0]
vvec[7]=vvec[1]
vvec[6]=vvec[2]
vvec[5]=vvec[3]
vvec[4]=1-2*(vvec[0]+vvec[1]+vvec[2]+vvec[3])
print("Indeterminates: ",tvec,vvec)
CHR95conds=CreateConditions(5,tvec,vvec)
#display(CHR95conds)

Indeterminates:  [a_0 + I*b_0, a_1 + I*b_1, a_2 + I*b_2, a_3 + I*b_3, a_4 + I*b_4, a_4 - I*b_4, a_3 - I*b_3, a_2 - I*b_2, a_1 - I*b_1, a_0 - I*b_0] [v_0, v_1, v_2, v_3, -2*v_0 - 2*v_1 - 2*v_2 - 2*v_3 + 1, v_3, v_2, v_1, v_0, 0]


In [ ]:
simpleCHR95conds=[it.simplify() for it in CHR95conds]
simpleCHR95conds[0],simpleCHR95conds[1] = simpleCHR95conds[1],simpleCHR95conds[0] # swap elements for easier indexing
res=[simpleCHR95conds[0]]+[(it.lhs-it.rhs).subs([
    ('a_0',S(0.048475520387300861784614942150004872732496275960676605704430)),
    ('a_1',S(0.150635519695238479617295632454033652021145596960357218146613)),
    ('a_2',S(0.062114931585048261469230020641774248967970727006873004431295)),
    ('a_3',S(0.03942548316294538780808735118455718625805068025688477432907649315)),
    ('a_4',S(0.1993485451694670093207720535696300400203367198152083973885855424)),
    ('b_0',S(0.004320853677325454041666651926455479652308652682908398349854853)),
    ('b_1',S(-0.06666435676733963925962676292404991504857683461279976061086538)),
    ('b_2',S(0.1283252146778869882406120485156541698482809759157383070985385043)),
    ('b_3',S(-0.1513147050159361009137685144878063897080171204514607870202000492)),
    ('b_4',S(0.09111504040429731652115356386419287657273864797837757794153296069)),
    ('v_0',S(0.1356385792611024462119279634247291695297982574967380761940065951)),
    ('v_1',S(0.06686127082929140828693929094576496971243553406915635491478201019)),
    ('v_2',S(0.1199647847544109515311819985684405286384441167551088741859806627)),
    ('v_3',S(1)/S(17)),
         ]).simplify() for it in simpleCHR95conds[1:]]
print(res)

[True, 0, 1.73472347597681e-18*I, -4.67354912939634e-17 - 1.0969574921618e-18*I, 3.26536183713281e-17, 2.44129643737065e-17 - 7.24183274621935e-19*I, 8.67361737988404e-18 - 8.67361737988404e-19*I, 2.31296463463574e-18 - 8.67361737988404e-19*I, 7.45931094670027e-18 + 3.46944695195361e-19*I, 2.19739643628234e-17 - 1.48541950238199e-18*I, 3.79470760369927e-18 - 1.02999206386123e-18*I, -9.25185853854297e-19 - 4.62592926927149e-19*I, 3.77302356024956e-18 - 4.20128341838133e-19*I, -2.37592741704831e-19 + 1.41454502191468e-19*I]


### CHR_15 6

In [ ]:
avec=[*symarray('a',16)]
bvec=[*symarray('b',16)]
tvec=[
    avec[0]+I*bvec[0],
    avec[1]+I*bvec[1],
    avec[2]+I*bvec[2],
    avec[3]+I*bvec[3],
    avec[4]+I*bvec[4],
    avec[5]+I*bvec[5],
    avec[6]+I*bvec[6],
    avec[7]+I*bvec[7],
    avec[7]-I*bvec[7],
    avec[6]-I*bvec[6],
    avec[5]-I*bvec[5],
    avec[4]-I*bvec[4],
    avec[3]-I*bvec[3],
    avec[2]-I*bvec[2],
    avec[1]-I*bvec[1],
    avec[0]-I*bvec[0]
]
vvec=[*symarray('v',16)]
vvec[15]=0
vvec[14]=vvec[0]
vvec[13]=vvec[1]
vvec[12]=vvec[2]
vvec[11]=vvec[3]
vvec[10]=vvec[4]
vvec[9]=vvec[5]
vvec[8]=vvec[6]
vvec[7]=1-2*(vvec[0]+vvec[1]+vvec[2]+vvec[3]+vvec[4]+vvec[5]+vvec[6])
print("Indeterminates: ",tvec,vvec)
CHR156conds=CreateConditions(6,tvec,vvec)
display(CHR156conds)

Indeterminates:  [a_0 + I*b_0, a_1 + I*b_1, a_2 + I*b_2, a_3 + I*b_3, a_4 + I*b_4, a_5 + I*b_5, a_6 + I*b_6, a_7 + I*b_7, a_7 - I*b_7, a_6 - I*b_6, a_5 - I*b_5, a_4 - I*b_4, a_3 - I*b_3, a_2 - I*b_2, a_1 - I*b_1, a_0 - I*b_0] [v_0, v_1, v_2, v_3, v_4, v_5, v_6, -2*v_0 - 2*v_1 - 2*v_2 - 2*v_3 - 2*v_4 - 2*v_5 - 2*v_6 + 1, v_6, v_5, v_4, v_3, v_2, v_1, v_0, 0]


[Eq(2*a_0 + 2*a_1 + 2*a_2 + 2*a_3 + 2*a_4 + 2*a_5 + 2*a_6 + 2*a_7, 1),
 True,
 Eq(-v_0*(a_0 + I*b_0) - v_0*(a_0 + 2*a_1 + 2*a_2 + 2*a_3 + 2*a_4 + 2*a_5 + 2*a_6 + 2*a_7 + I*b_0) - v_1*(a_0 + a_1 + I*b_0 + I*b_1) - v_1*(a_0 + a_1 + 2*a_2 + 2*a_3 + 2*a_4 + 2*a_5 + 2*a_6 + 2*a_7 + I*b_0 + I*b_1) - v_2*(a_0 + a_1 + a_2 + I*b_0 + I*b_1 + I*b_2) - v_2*(a_0 + a_1 + a_2 + 2*a_3 + 2*a_4 + 2*a_5 + 2*a_6 + 2*a_7 + I*b_0 + I*b_1 + I*b_2) - v_3*(a_0 + a_1 + a_2 + a_3 + I*b_0 + I*b_1 + I*b_2 + I*b_3) - v_3*(a_0 + a_1 + a_2 + a_3 + 2*a_4 + 2*a_5 + 2*a_6 + 2*a_7 + I*b_0 + I*b_1 + I*b_2 + I*b_3) - v_4*(a_0 + a_1 + a_2 + a_3 + a_4 + I*b_0 + I*b_1 + I*b_2 + I*b_3 + I*b_4) - v_4*(a_0 + a_1 + a_2 + a_3 + a_4 + 2*a_5 + 2*a_6 + 2*a_7 + I*b_0 + I*b_1 + I*b_2 + I*b_3 + I*b_4) - v_5*(a_0 + a_1 + a_2 + a_3 + a_4 + a_5 + I*b_0 + I*b_1 + I*b_2 + I*b_3 + I*b_4 + I*b_5) - v_5*(a_0 + a_1 + a_2 + a_3 + a_4 + a_5 + 2*a_6 + 2*a_7 + I*b_0 + I*b_1 + I*b_2 + I*b_3 + I*b_4 + I*b_5) - v_6*(a_0 + a_1 + a_2 + a_3 + a_4 + a_5 + 

In [ ]:
simpleCHR156conds=CHR156conds#[it.simplify() for it in CHR156conds]# The simplification probably will take ages...
simpleCHR156conds[0],simpleCHR156conds[1] = simpleCHR156conds[1],simpleCHR156conds[0] # swap elements for easier indexing
res=[simpleCHR156conds[0]]+[(it.lhs-it.rhs).subs([
    ('a_0',S(0.018407829100474781904003178937442)),
    ('a_1',S(0.037640136069562677570517610827614)),
    ('a_2',S(0.081602637043566199583289018818188)),
    ('a_3',S(0.08163394645009567118137080903697)),
    ('a_4',S(0.082577837091366965137963780758131)),
    ('a_5',S(0.096825373607680378408689859741856)),
    ('a_6',S(0.028447864413379336326027072059858)),
    ('a_7',S(0.07286437622387398988813866981994)),
    ('b_0',S(0.060063925504056716987384011897458)),
    ('b_1',S(-0.078183261287778862724569177453338)),
    ('b_2',S(0.044188227706548767875286813464238)),
    ('b_3',S(-0.10257156270865030554005352335570)),
    ('b_4',S(0.077099467183537374713948506168652)),
    ('b_5',S(0.035803350489900133270555458990013)),
    ('b_6',S(-0.087263167058591631392917553485744)),
    ('b_7',S(0.14749079765627217175324647517686)),
    ('v_0',S(0.013599999999999999242272785693331)),
    ('v_1',S(0.10132124738923978821039934776422)),
    ('v_2',S(0.08496255121091424240063136564739)),
    ('v_3',S(0.023858373653897122447214292182343)),
    ('v_4',S(0.12203432116901233344250458684336)),
    ('v_5',S(0.09460538089501579698109829739963)),
    ('v_6',S(0.058633436576191442147232738898912)),
         ]).simplify() for it in simpleCHR156conds[1:]]
print(res)

[True, -1.11022302462516e-16, 2.77555756156289e-17 + 5.77337656848531e-18*I, 6.93889390390723e-17 + 5.78241158658936e-18*I, 3.46944695195361e-17 + 3.90312782094782e-18*I, 2.58853268680914e-17 - 4.44522890719057e-19*I, 4.39463280580791e-17 - 1.73472347597681e-18*I, 2.35746209879817e-17 + 2.76217444099627e-18*I, 2.6599093298311e-17 + 2.89120579329468e-18*I, 7.92009687000661e-18 + 1.94885340504269e-18*I, 4.62592926927149e-19*I, 2.33817233514603e-17 + 1.78159263239155e-18*I, 7.88178274635914e-18 + 1.45847603279363e-18*I, 1.19082707693257e-17 + 4.19466597348091e-19*I, 5.81855165900554e-18 + 7.878535786728e-19*I, 1.51426903423809e-18 + 5.20417042793042e-19*I, 8.83963583754588e-18 + 7.86046575051991e-19*I, 7.59995606185325e-19 + 5.78542325929071e-19*I, 1.3588667228485e-18 + 4.90714420775991e-19*I, 1.00074532262092e-17 + 1.02308851786644e-19*I, 8.14356298444668e-19 + 1.30104260698261e-19*I, 4.33680868994202e-19 + 1.68446618777305e-19*I, 1.00786998998776e-19 + 1.82126450268263e-22*I]
